In [1]:
#라이브러리 import
#python == 3.7.0

import random
import pandas as pd
import numpy as np
import os
import cv2

import tensorflow as tf # tensorflow == 2.6.0
from tensorflow.keras import optimizers, layers, models
from tensorflow_addons.metrics import F1Score # tfa == 0.15.0, F1score를 위한 별도 설치 라이브러리, 선택 사항 (필수 x)
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.utils import compute_class_weight
from tqdm.auto import tqdm

from sklearn.model_selection import train_test_split

C:\Users\WBH\anaconda3\envs\pytorchstudy\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CFG = {
    'VIDEO_LENGTH':50, # 10프레임 * 5초
    'IMG_SIZE':128,
    'EPOCHS':10,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':4,
    'SEED':41
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything(CFG['SEED']) # Seed 고정

In [3]:
def frames_from_video_file(video_path): # video를 읽어서 50fps에 해당하는 사진으로 반환
    frames = []
    cap = cv2.VideoCapture(video_path)
    for _ in range(CFG['VIDEO_LENGTH']):
        _, img = cap.read()
        img = cv2.resize(img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
        img = img / 255.
        frames.append(img)

    result = np.array(frames)[..., [2,1,0]]

    return result


class FrameGenerator: # 모든 video를 ram에 적재할 수 없기 때문에 batch 마다 generate 해주기 위한 generator 생성
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __call__(self):
        for tp in self.dataframe.itertuples():
            video_frames = frames_from_video_file(tp.video_path)
            label = tp.label # Encode labels
            label = to_categorical(label, 13)

            yield video_frames, label

In [12]:
df = pd.read_csv('./train.csv')

train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, stratify=df['label'], random_state=CFG['SEED'])

weights = compute_class_weight(class_weight='balanced', classes=np.unique(train.label), y=train.label)
# unbalance 한 데이터이기 때문에 class weight 계산

In [13]:
# def to_gif(images): # 변환된 이미지 gif로 저장해서 확인
#     import imageio

#     converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
#     imageio.mimsave('./animation.gif', converted_images, fps=10)


In [14]:
# model에 주입하기 위한 dataset 객체 생성

output_signature = (tf.TensorSpec(shape=(None, None, None, 3), dtype=tf.float32),
                    tf.TensorSpec(shape=(13), dtype=tf.int16))

train_ds = tf.data.Dataset.from_generator(FrameGenerator(train),
                                          output_signature=output_signature)

val_ds = tf.data.Dataset.from_generator(FrameGenerator(val),
                                          output_signature=output_signature)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size = AUTOTUNE)
train_ds = train_ds.batch(CFG['BATCH_SIZE'])
val_ds = val_ds.cache().prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.batch(CFG['BATCH_SIZE'])

train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

val_frames, val_labels = next(iter(val_ds))
print(f'Shape of validation set of frames: {val_frames.shape}')
print(f'Shape of validation labels: {val_labels.shape}')

Shape of training set of frames: (4, 50, 128, 128, 3)
Shape of training labels: (4, 13)
Shape of validation set of frames: (4, 50, 128, 128, 3)
Shape of validation labels: (4, 13)


In [16]:
model = models.Sequential()
model.add(layers.Conv3D(32, (1, 3, 3), activation='relu', input_shape=(50, 128, 128, 3)))
model.add(layers.TimeDistributed(layers.BatchNormalization()))
model.add(layers.MaxPooling3D((1, 2, 2)))

model.add(layers.Conv3D(64, (1, 2, 2), activation='relu'))
model.add(layers.TimeDistributed(layers.BatchNormalization()))
model.add(layers.MaxPooling3D((1, 2, 2)))

model.add(layers.Conv3D(128, (1, 2, 2), activation='relu'))
model.add(layers.TimeDistributed(layers.BatchNormalization()))
model.add(layers.MaxPooling3D((1, 2, 2)))

model.add(layers.Conv3D(256, (1, 2, 2), activation='relu'))
model.add(layers.TimeDistributed(layers.BatchNormalization()))
model.add(layers.MaxPooling3D((1, 2, 2)))

model.add(layers.Conv3D(512, (1, 2, 2), activation='relu'))
model.add(layers.TimeDistributed(layers.BatchNormalization()))
model.add(layers.MaxPooling3D((1, 2, 2)))

model.add(layers.Conv3D(1024, (1, 2, 2), activation='relu'))
model.add(layers.TimeDistributed(layers.BatchNormalization()))
model.add(layers.GlobalAveragePooling3D())

model.add(layers.Dense(13, activation='softmax'))


model.compile(optimizer=optimizers.Adam(CFG['LEARNING_RATE']),
              loss='categorical_crossentropy',
              metrics=['accuracy',
                       F1Score(num_classes=13, average='macro') # 선택 사항, tensorflow-addons 가 없다면 'accuracy'만 적용
                       ])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 50, 126, 126, 32)  896       
_________________________________________________________________
time_distributed (TimeDistri (None, 50, 126, 126, 32)  128       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 50, 63, 63, 32)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 50, 62, 62, 64)    8256      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 62, 62, 64)    256       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 50, 31, 31, 64)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 50, 30, 30, 128)   3

In [17]:
es = EarlyStopping(monitor='val_accuracy', patience=5, mode='max', restore_best_weights=True)
history = model.fit(train_ds, epochs=50, validation_data=val_ds, callbacks=[es],
                    class_weight={i:weights[i] for i in range(len(weights))})

Epoch 1/50
540/540 [==============================] - 324s 592ms/step - loss: 3.0808 - accuracy: 0.0987 - f1_score: 0.0615 - val_loss: 6.4994 - val_accuracy: 0.0296 - val_f1_score: 0.0227
Epoch 2/50
540/540 [==============================] - 67s 123ms/step - loss: 1.6719 - accuracy: 0.1951 - f1_score: 0.1448 - val_loss: 4.0721 - val_accuracy: 0.1778 - val_f1_score: 0.1061
Epoch 3/50
540/540 [==============================] - 67s 124ms/step - loss: 1.0388 - accuracy: 0.3221 - f1_score: 0.2280 - val_loss: 3.3856 - val_accuracy: 0.1500 - val_f1_score: 0.1124
Epoch 4/50
540/540 [==============================] - 67s 124ms/step - loss: 0.6533 - accuracy: 0.4690 - f1_score: 0.3415 - val_loss: 2.4264 - val_accuracy: 0.3722 - val_f1_score: 0.1616
Epoch 5/50
540/540 [==============================] - 67s 124ms/step - loss: 0.4712 - accuracy: 0.5811 - f1_score: 0.4417 - val_loss: 2.2698 - val_accuracy: 0.4111 - val_f1_score: 0.1535
Epoch 6/50
540/540 [==============================] - 67s 124ms/

In [20]:
test_df = pd.read_csv('./test.csv')
test_df['label'] = 0

test_ds = tf.data.Dataset.from_generator(FrameGenerator(test_df),
                                          output_signature=output_signature)

test_ds = test_ds.cache().prefetch(buffer_size = AUTOTUNE)
test_ds = test_ds.batch(CFG['BATCH_SIZE'])

pred = model.predict(test_ds)
pred = np.argmax(pred, axis=1)

submit = pd.read_csv('./sample_submission.csv')
submit['label'] = pred
submit.to_csv('./submission.csv', index=False)
print('pred and save done.')

# 참고한 사이트
# https://www.tensorflow.org/tutorials/video/video_classification
# https://www.tensorflow.org/tutorials/load_data/video?hl=ko
# https://bestkcs1234.tistory.com/61  # tensorflow-addons 설치 방법

pred and save done.
